# Development Right Transfers
#### Meeting Agenda: Development Rights Transfer & Conversion Analysis

1. Objectives & Data Review
    * Goal: Standardize ETL of transfer → conversion process & confirm data integration
    * Review analysis requirements
    * Review of data sources:
        * LT Info TDR Transactions (APN, type, land capability, quantity)
        * Parcel Master (jurisdiction, town center proximity, local plan)
        * Accela (transfer status, permit data)
2. Coding Plan & Standardization
    * Walkthrough of data integration approach
    * Confirm transfer first, then convert process
    * Address any inconsistencies or edge cases
3. Key Analyses
    * Land Capability: SEZ, sensitive, non-sensitive
    * Distance from Center: Trends by proximity
    * Interjurisdictional Activity: Transfers & conversions across boundaries
4. Next Steps
    * Assign action items & timeline for completion


## Notes: From Ken
- Transfer Reporting Status - comes from LTinfo
    - Transfers come out of LTinfo 
- Status of Transaction - comes from Accela
    - transaction is considered complete and development rights are moved to recieving parcel when the transfer is acknowledged
- Status of the Development on the Recieving Parcel
    - associate the transaction in LTinfo to the development project in Accela/Local Jurisdiction data
    - what is the status of the development project? (i.e. when is it existing on the ground)
 
- Transfer vs Conversion sequence
    - should be transfer dev rights then convert on the recieving parcel 
- Parcel Geneology Lookup needs to be built
    - Identify old APNs and current APNs
- Data Clean-up
    - categorization of unit types has evolved (e.g. PRUU vs RUU) same/same now
- Conversions
    - we track the transfers and then convert onto the recieving parcel (or onsite conversion)
    - track conversion net change
- 

## Setup

### Packages

In [ ]:
import pandas as pd
import os
import pathlib
import arcpy
from arcgis.features import FeatureLayer, GeoAccessor, GeoSeriesAccessor
from utils import *
from datetime import datetime
from time import strftime  

### Global Variables

In [ ]:
# set data frame display options
# pandas options
pd.options.mode.copy_on_write = True
pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows    = 999
pd.options.display.float_format = '{:,.2f}'.format
   
# set environement workspace to in memory 
arcpy.env.workspace = 'memory'
# overwrite true
arcpy.env.overwriteOutput = True
# Set spatial reference to NAD 1983 UTM Zone 10N
sr = arcpy.SpatialReference(26910)
arcpy.env.outputCoordinateSystem = sr

# current working directory
local_path = pathlib.Path().absolute()
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir   = local_path.parents[0] / 'Reporting/data/raw_data'
# folder to save processed data
out_dir    = local_path.parents[0] / 'Reporting/data/processed_data'

# network path to connection files
filePath = "F:/GIS/PARCELUPDATE/Workspace/"
# database file path 
sdeBase    = os.path.join(filePath, "Vector.sde")
sdeCollect = os.path.join(filePath, "Collection.sde")
sdeTabular = os.path.join(filePath, "Tabular.sde")
sdeEdit    = os.path.join(filePath, "Edit.sde")

## Data Processing
### Data Pipeline Overview
1. Extract data from LT Info, Parcel Master, and Accela.
2. Clean and preprocess data for consistency.
3. Merge datasets using APN as the primary key.
4. Standardize workflow: **transfer first, then convert**.
5. Identify and resolve inconsistencies.

### 1. Extract data from LT Info, Parcel Master, and Accela.

#### Data Sources
- **LT Info TDR Transactions**: Tracks APN, development right type, land capability, and quantity.
- **Parcel Master**: Provides jurisdiction, town center proximity, and
- **Accela**: Contains transfer status and permit details.

> Sources
* https://www.laketahoeinfo.org/WebServices/List
* https://maps.trpa.org/server/rest/services/
* https://parcels.laketahoeinfo.org/TdrTransaction/TransactionList
* sdeBase, sdeCollect, sdeTabular

#### 1.1 Parcel Master

In [ ]:
# web service and database paths
# portal_ParcelMaster = 'https://maps.trpa.org/server/rest/services/Parcel_Master/FeatureServer/0'
sde_ParcelMaster    = Path(sdeBase) / "sde.SDE.Parcels\\sde.SDE.Parcel_Master"
# get spatially enabled dataframes
sdfParcels = pd.DataFrame.spatial.from_featureclass(sde_ParcelMaster)

#### 1.2 LTInfo Data

In [ ]:
# transfer grid downloaded from LTinfo https://parcels.laketahoeinfo.org/TdrTransaction/TransactionList
# dfTransactionsGrid = pd.read_csv(local_path / "data/raw_data/TransactedAndBankedDevelopmentRights.csv")

# grid path
# dfTransactionsGrid = pd.read_excel(local_path / "data/raw_data/TdrTransactions as of 02_06_2025 12_00 PM.xlsx")
dfTransfers   = pd.read_excel(data_dir / "TdrTransactions as of 02_06_2025 12_00 PM.xlsx", sheet_name='Transfers')
dfConversions = pd.read_excel(data_dir / "TdrTransactions as of 02_06_2025 12_00 PM.xlsx", sheet_name='Conversions') 
dfConvTransfer = pd.read_excel(data_dir / "TdrTransactions as of 02_06_2025 12_00 PM.xlsx", sheet_name='Conversion with Transfers')

In [ ]:
dfTransfers.info()

In [ ]:
## LT Info Data
# get banked
dfDevRightBanked     = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# Verified Development Rights from Accela as a DataFrame
dfDevRightForAccela  = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetParcelDevelopmentRightsForAccela/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# Development Rights Transacted and Banked as a DataFrame
dfDevRightTransacted = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetTransactedAndBankedDevelopmentRights/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")
# All Parcels as a DataFrame
dfLTParcel           = pd.read_json("https://www.laketahoeinfo.org/WebServices/GetAllParcels/JSON/e17aeb86-85e3-4260-83fd-a2b32501c476")

In [ ]:
dfDevRightTransacted.info()

#### 1.3 Accela Permit Data

In [ ]:
# API access to download excel file of Accela Record Details
accelaRecorDetails = "https://laketahoeinfo.org/Api/GetAccelaRecordDetailsExcel/1A77D078-B83E-44E0-8CA5-8D7429E1A6B4"
# download the file
dfAccelaRecord = pd.read_excel(accelaRecorDetails)

In [ ]:
# get detailed project data report
dfDetailedProjectData = pd.read_excel(data_dir / "PermitStatusReport.xlsx")
dfDetailedProjectData.info()

### 2. Clean and preprocess data for consistency.

In [ ]:
final_schema = ['Transaction Status',
                'Transaction Type',
                'Development Right',
                'Sending Parcel APN',
                'Receiving Parcel APN',
                'Sending Quantity',
                'Receiving Quantity',
                'Sending Bailey Rating',
                'Receiving Bailey Rating',
                'Issued',
                'Acknowledged',
                'Project Completed', 
                'JURISDICTION',  
                'PLAN_TYPE',
                'LOCATION_TO_TOWNCENTER']

### 3. Merge datasets using APN as the primary key.

In [ ]:
# filter columns in sdf Parcels
parcels = sdfParcels[['APN', 'JURISDICTION', 'PLAN_TYPE', 'LOCATION_TO_TOWNCENTER']]

In [88]:
# merge dfTransfers with dfDetailedProjectData
df = pd.merge(dfTransfers, dfDetailedProjectData, left_on='Accela ID', right_on='File Number', how='left')
# merge Sending APN to Parcel APN
df = pd.merge(df, parcels, left_on='Sending Parcel APN', right_on='APN', how='left')
# rename parcels fields with prefix SENDING_
df.rename(columns={'JURISDICTION': 'SENDING_JURISDICTION', 'PLAN_TYPE': 'SENDING_PLAN_TYPE', 'LOCATION_TO_TOWNCENTER': 'SENDING_LOCATION_TO_TOWNCENTER'}, inplace=True)
# merge Receiving APN to Parcel APN
df = pd.merge(df, parcels, left_on='Receiving Parcel APN', right_on='APN', how='left')
# rename parcels fields with prefix RECEIVING_
df.rename(columns={'JURISDICTION': 'RECEIVING_JURISDICTION', 'PLAN_TYPE': 'RECEIVING_PLAN_TYPE', 'LOCATION_TO_TOWNCENTER': 'RECEIVING_LOCATION_TO_TOWNCENTER'}, inplace=True)

# 

In [89]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 43 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   Transaction ID                    505 non-null    int64         
 1   Transaction Transfer ID           505 non-null    int64         
 2   Transaction Number                505 non-null    object        
 3   Lead Agency                       505 non-null    object        
 4   Transaction Status                505 non-null    object        
 5   Transaction Type                  505 non-null    object        
 6   Development Right                 505 non-null    object        
 7   Accela ID                         499 non-null    object        
 8   Created By                        505 non-null    object        
 9   Created                           505 non-null    datetime64[ns]
 10  Updated By                        505 non-null    

### 4. Standardize workflow: **transfer first, then convert**.

### 5. Identify and resolve inconsistencies.

In [ ]:
# APN changes over time

# Development Right name changed (e.g. RDR changed to PRUU)

# CTC and NDSL transactions will show up in the system later

# can be multiple transactions for the same APN and it could be the same development type
# transaction ID is unique so the many to many APN to transaction ID relationship can be used to join the data

## Key Analyses & Insights

### Land Capability Analysis
- Categorize transfers by SEZ, sensitive, and non-sensitive land.

In [92]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 505 entries, 0 to 504
Data columns (total 45 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   Transaction ID                      505 non-null    int64         
 1   Transaction Transfer ID             505 non-null    int64         
 2   Transaction Number                  505 non-null    object        
 3   Lead Agency                         505 non-null    object        
 4   Transaction Status                  505 non-null    object        
 5   Transaction Type                    505 non-null    object        
 6   Development Right                   505 non-null    object        
 7   Accela ID                           499 non-null    object        
 8   Created By                          505 non-null    object        
 9   Created                             505 non-null    datetime64[ns]
 10  Updated By                

In [ ]:
df = df.copy()
# categorize
landcap_dict = {'1b':'SEZ',
                '1a':'Sensitive',
                '2':'Sensitive',
                '3':'Sensitive',
                '4':'Non-Sensitive',
                '5':'Non-Sensitive',
                '6':'Non-Sensitive',
                '7':'Non-Sensitive'}
# map land capability to land capability category
df['Sending_Land_Capability_Category'] = df['Sending Bailey Rating'].map(landcap_dict)
# map land capability to land capability category
df['Receiving_Land_Capability_Category'] = df['Receiving Bailey Rating'].map(landcap_dict)


In [ ]:
# groupby sending_land_capability_category and receiving_land_capability_category
df_landcap_group = df.groupby(['Sending_Land_Capability_Category', 'Receiving_Land_Capability_Category', 'Development Right']).agg({
                                'Sending Quantity':'sum',
                                'Receiving Quantity': 'sum'}).reset_index()
df_landcap_group

,Sending_Land_Capability_Category,Receiving_Land_Capability_Category,Development Right,Sending Quantity,Receiving Quantity
0,Non-Sensitive,Non-Sensitive,Commercial Floor Area (CFA),70642,70642
1,Non-Sensitive,Non-Sensitive,Coverage (hard),26392,26392
2,Non-Sensitive,Non-Sensitive,Coverage (potential),32756,32756
3,Non-Sensitive,Non-Sensitive,Multi-Family Residential Unit of Use (MFRUU),1,1
4,Non-Sensitive,Non-Sensitive,Potential Residential Unit of Use (PRUU),12,12
5,Non-Sensitive,Non-Sensitive,Single-Family Residential Unit of Use (SFRUU),59,59
6,Non-Sensitive,Non-Sensitive,Tourist Accommodation Unit (TAU),249,249
7,SEZ,Non-Sensitive,Coverage (hard),172080,141417
8,SEZ,Non-Sensitive,Restoration Credit,62,62
9,SEZ,Non-Sensitive,Single-Family Residential Unit of Use (SFRUU),16,16


### Proximity Analysis
- Assess distance of transfers from town centers.

In [95]:
df_towncenter = df.groupby(['SENDING_LOCATION_TO_TOWNCENTER', 'RECEIVING_LOCATION_TO_TOWNCENTER', 'Development Right']).agg({
                                'Sending Quantity':'sum',
                                'Receiving Quantity': 'sum'}).reset_index() 
df_towncenter

,SENDING_LOCATION_TO_TOWNCENTER,RECEIVING_LOCATION_TO_TOWNCENTER,Development Right,Sending Quantity,Receiving Quantity
0,Outside Buffer,Further than Quarter Mile from Town Center,Potential Residential Unit of Use (PRUU),1,1
1,Outside Buffer,Outside Buffer,Commercial Floor Area (CFA),5340,5340
2,Outside Buffer,Outside Buffer,Coverage (hard),46645,44145
3,Outside Buffer,Outside Buffer,Coverage (potential),60464,60464
4,Outside Buffer,Outside Buffer,Coverage (soft),1118,1118
5,Outside Buffer,Outside Buffer,Potential Residential Unit of Use (PRUU),18,18
6,Outside Buffer,Outside Buffer,Restoration Credit,2798,2798
7,Outside Buffer,Outside Buffer,Single-Family Residential Unit of Use (SFRUU),29,29
8,Outside Buffer,Quarter Mile Buffer,Coverage (hard),9057,9057
9,Outside Buffer,Quarter Mile Buffer,Coverage (potential),18220,18220


### Interjurisdictional Transfers
- Examine development right transfers across jurisdictional boundaries.

In [96]:
# 
df_jurisdiction = df.groupby(['SENDING_JURISDICTION', 'RECEIVING_JURISDICTION', 'Development Right']).agg({
                                'Sending Quantity':'sum',
                                'Receiving Quantity': 'sum'}).reset_index() 
df_jurisdiction

,SENDING_JURISDICTION,RECEIVING_JURISDICTION,Development Right,Sending Quantity,Receiving Quantity
0,CSLT,CSLT,Commercial Floor Area (CFA),37339,37339
1,CSLT,CSLT,Coverage (hard),164892,134229
2,CSLT,CSLT,Coverage (potential),30836,30836
3,CSLT,CSLT,Multi-Family Residential Unit of Use (MFRUU),14,14
4,CSLT,CSLT,Potential Residential Unit of Use (PRUU),38,38
5,CSLT,CSLT,Restoration Credit,1208,1208
6,CSLT,CSLT,Single-Family Residential Unit of Use (SFRUU),56,56
7,CSLT,CSLT,Tourist Accommodation Unit (TAU),40,40
8,CSLT,DG,Potential Residential Unit of Use (PRUU),3,3
9,CSLT,DG,Single-Family Residential Unit of Use (SFRUU),3,3


## Next Steps
### Action Items
- Build Accela Report that gets us issued data 
- Get Accela ID and Jurisdiction Permit Number into LTinfo Web Service Development Rights Transacted and Banked
- Fix Parcel geneology for current 'APN', 'Recieving APN' and 'Sending APN' 